### Import Library

In [ ]:
import pandas as pd 
import numpy as np 
import os 

### Download the dataset from kaggle

In [ ]:
{"username":"nesreenahmed","key":"c53f3958a0f70dff37202dc3f20b134a"}

{'username': 'nesreenahmed', 'key': 'c53f3958a0f70dff37202dc3f20b134a'}

In [ ]:
# to download the dataset from kaggle
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"nesreenahmed","key":"c53f3958a0f70dff37202dc3f20b134a"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c dogs-vs-cats

100% 809M/812M [00:08<00:00, 130MB/s]
100% 812M/812M [00:08<00:00, 98.0MB/s]


In [ ]:
os.listdir()

['.config', 'dogs-vs-cats.zip', 'sample_data']

In [ ]:
import zipfile # the zipfile module used to extract the data from zipped files

In [ ]:
zip_file=['test1','train']
for zip_file in zip_file:
  with zipfile.ZipFile('/content/dogs-vs-cats.zip'.format(zip_file),'r') as z: # from the variable called zipfile used the method called ZipFile and perform read operation
    z.extractall('.') # extract all the files seperated by the "."
    print('{} unzipped'.format(zip_file))

test1 unzipped
train unzipped


In [ ]:
os.listdir() # to list all the directories within the current directory

['.config',
 'dogs-vs-cats.zip',
 'test1.zip',
 'train.zip',
 'sampleSubmission.csv',
 'sample_data']

In [ ]:
with zipfile.ZipFile('/content/train.zip','r') as z:
  z.extractall('.') 
  print('{}unzipped'.format(zip_file))

trainunzipped


In [ ]:
with zipfile.ZipFile('/content/test1.zip') as z:
  z.extractall('.')
  print('{}unzipped'.format(zip_file))

trainunzipped


### Prepare Traning Data

In [ ]:
img_file='/content/train'
file_names=os.listdir(img_file)
categories =[]
path=[]
for file_name in file_names:
  category=file_name.split('.')[0]
  full_path=os.path.join(img_file,file_name)
  path.append(full_path)
  categories .append(category)

df=pd.DataFrame()
df['img_path']=path
df['category']=categories 
df.head(10)

,img_path,category
0,/content/train/cat.11580.jpg,cat
1,/content/train/dog.7796.jpg,dog
2,/content/train/cat.759.jpg,cat
3,/content/train/dog.6413.jpg,dog
4,/content/train/dog.3618.jpg,dog
5,/content/train/dog.3109.jpg,dog
6,/content/train/cat.992.jpg,cat
7,/content/train/dog.7640.jpg,dog
8,/content/train/cat.10122.jpg,cat
9,/content/train/cat.3017.jpg,cat


In [ ]:
df.category.value_counts()

cat    12500
dog    12500
Name: category, dtype: int64

### Split train and test data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_val=train_test_split(df,test_size=0.1,random_state=42)

In [ ]:
x_train.shape

(22500, 2)

In [ ]:
x_val.shape

(2500, 2)

### Build Model

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from keras.layers import BatchNormalization

In [ ]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(250,250,3)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.1))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.1))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.1))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5)) ### increase it to 0.5
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

### Traning Generator

In [ ]:
img_pre=ImageDataGenerator(rescale=1/255,width_shift_range=0.1,height_shift_range=0.2)
img_pree=img_pre.flow_from_dataframe(dataframe=x_train,x_col='img_path',y_col='category',target_size=(250,250),class_mode='binary')

Found 22500 validated image filenames belonging to 2 classes.


### Validation Generator

In [ ]:
val_pre=ImageDataGenerator(rescale=1/255,width_shift_range=0.1,height_shift_range=0.2)
val_pree=val_pre.flow_from_dataframe(dataframe=x_val,x_col='img_path',y_col='category',target_size=(250,250),class_mode='binary')

Found 2500 validated image filenames belonging to 2 classes.


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 248, 248, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 246, 246, 32)      9248      
                                                                 
 batch_normalization (BatchN  (None, 246, 246, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 123, 123, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 123, 123, 32)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 121, 121, 64)      1

### Training the model

In [ ]:
history=model.fit(img_pree,epochs=50,batch_size=64,validation_data=val_pree,verbose=1,steps_per_epoch=x_train.shape[0]//64,validation_steps=x_val.shape[0]//64)

Epoch 1/50
351/351 [==============================] - 208s 559ms/step - loss: 0.7798 - accuracy: 0.5978 - val_loss: 0.8340 - val_accuracy: 0.5264
Epoch 2/50
351/351 [==============================] - 193s 550ms/step - loss: 0.6280 - accuracy: 0.6620 - val_loss: 0.7404 - val_accuracy: 0.6218
Epoch 3/50
351/351 [==============================] - 196s 557ms/step - loss: 0.5563 - accuracy: 0.7164 - val_loss: 0.7626 - val_accuracy: 0.5905
Epoch 4/50
351/351 [==============================] - 195s 555ms/step - loss: 0.4944 - accuracy: 0.7635 - val_loss: 0.7020 - val_accuracy: 0.6370
Epoch 5/50
351/351 [==============================] - 198s 562ms/step - loss: 0.4394 - accuracy: 0.7989 - val_loss: 0.7661 - val_accuracy: 0.6106
Epoch 6/50
351/351 [==============================] - 193s 549ms/step - loss: 0.3836 - accuracy: 0.8304 - val_loss: 0.5399 - val_accuracy: 0.7380
Epoch 7/50
351/351 [==============================] - 193s 549ms/step - loss: 0.3566 - accuracy: 0.8405 - val_loss: 0.3977 -

### Saving the model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model.save_weights('/content/gdrive')

In [ ]:
model.save('final_model.h5')